In [28]:
!pip install cmapPy

In [27]:
import random
from cmapPy.pandasGEXpress.parse import parse
from matplotlib import colors as mcolors
import matplotlib.pyplot as plt

In [26]:
#uploading the file 
from google.colab import files
uploaded = files.upload()

Saving PAAD.gct to PAAD (1).gct


In [29]:
#loading file using cmapPy 
import io

In [35]:
#Renaming the dataframes for convenience.
data_mat = parse('PAAD.gct',).data_df                     
data_col_m = parse('PAAD.gct',col_meta_only=True).T
# saving sample names in list.
samp = list(finaldata_mat.columns.values.tolist())
print("Sample Names are: ", samp[:5])

Sample Names are:  ['aab1-Primary solid Tumor', 'aab4-Primary solid Tumor', 'aab6-Primary solid Tumor', 'aab8-Primary solid Tumor', 'aab9-Primary solid Tumor']


In [31]:
#dataframe dimension original
print("Data Dimension: {}".format(str(data_mat.shape)))


#Removing NA values and cleaning of data for further calculations.
finaldata_mat = data_mat.dropna()

#dataframe dimension after cleaning.
print("New Data Dimension: {}".format(str(finaldata_mat.shape)))

Data Dimension: (18465, 183)
New Data Dimension: (14098, 183)


In [32]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

#Creating array for PCA Technique for dimensionality reduction
val = finaldata_mat.values.T              #transpoing the matrix 

print("After transpoing: ", val.shape)

#standarizing the values
scaler = StandardScaler()
val = scaler.fit_transform(val)

After transpoing:  (183, 14098)


In [33]:
#using PCA dimensionality reduction 
from sklearn.decomposition import PCA

pca = PCA(n_components=2)

pval = pca.fit_transform(val)

In [36]:
#for seperating outliers from the data.
import pandas as pd
out = pd.DataFrame(pval, columns=['Val1', 'Val2'], index=samp)
out.head()

,Val1,Val2
aab1-Primary solid Tumor,23.225243,-14.563448
aab4-Primary solid Tumor,4.242101,-14.212594
aab6-Primary solid Tumor,28.721016,-27.624334
aab8-Primary solid Tumor,-0.725019,-24.624250
aab9-Primary solid Tumor,0.321680,2.133417


In [37]:
#storing outliers (neuroendocrine tumors) in a variable
indexNames = out[ (out['Val1'] < -100) | (out['Val1'] > 100) | (out['Val2'] > 50)].index
indexNames

Index(['aabv-Primary solid Tumor', 'aaqm-Primary solid Tumor',
       'a9ij-Primary solid Tumor', 'a9il-Primary solid Tumor',
       'a9in-Primary solid Tumor', 'a9io-Primary solid Tumor',
       'a9ir-Primary solid Tumor', 'a9is-Primary solid Tumor',
       'a9iv-Primary solid Tumor', '6880-Primary solid Tumor',
       'a45n-Solid Tissue Normal', 'a7ol-Primary solid Tumor',
       'a7op-Primary solid Tumor', '8519-Primary solid Tumor',
       '7897-Primary solid Tumor', 'a75w-Primary solid Tumor'],
      dtype='object')

In [38]:
print('Number of Neuroendocrine Tumors are: {}'.format(len(indexNames)))

Number of Neuroendocrine Tumors are: 16


In [39]:
#dropping neuroendocrine from the dataframe
out.drop(indexNames, inplace=True)

#new info
out.info()

<class 'pandas.core.frame.DataFrame'>
Index: 167 entries, aab1-Primary solid Tumor to aapl-Primary solid Tumor
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Val1    167 non-null    float32
 1   Val2    167 non-null    float32
dtypes: float32(2)
memory usage: 2.6+ KB


In [40]:
#uploading IFN file
from google.colab import files
uploaded = files.upload()

Saving type1_IFN.txt to type1_IFN (1).txt


In [41]:
import pandas as pd
#reading  Type 1 IFN signature
ifn_sig = pd.read_csv('type1_IFN.txt',header=None)
ifn_sig.columns = ['member']
ifn_sig.head()

,member
0,IFIT1
1,IFI44
2,IFIT3
3,MX2
4,OAS1


In [50]:
ifn_sign = finaldata_mat.loc[ifn_sig]  
#or

ValueError: ignored

In [48]:
# dataset containing interferons gene samples only. error in .loc
ifn_sig_data = pval[ifn_sig['member'].to_list()]
ifn_sig_data.shape

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  


IndexError: ignored

In [50]:
#PCA of interferon genes
X_std = StandardScaler().fit_transform(ifn_sig_data)
n_components = 2
sklearn_pca = sklearnPCA(n_components=n_components)
X_reduced  = sklearn_pca.fit_transform(X_std)
fig = plt.figure(1, figsize=(10,6))
sns.scatterplot(X_reduced[:, 0],  X_reduced[:, 1],hue=imputed_my_data['sample_type'])

In [43]:
!pip install gsva
from GSVA import gsva
from plotnine import *
from sklearn.manifold import TSNE

In [50]:
#plotting tsne clusters of interferon gene ids.
XV = TSNE(n_components=2).\
    fit_transform(ifn_sig_data.T)
df = pd.DataFrame(XV).rename(columns={0:'x',1:'y'})
(ggplot(df,aes(x='x',y='y'))
 + geom_point(alpha=0.2)
)

In [ ]:
#GSVA of expression file and interferon file.
pathways_df = gsva(imputed_my_data,ifn_sig)